In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [ ]:
df=pd.read_csv("F:\\abc.csv")


In [ ]:
RANDOM_STATE = 13



In [ ]:
def fill_ses(x):
    ses = 0
    if x > 17:
        ses = 1.0
    elif (x < 17) and (x > 15):
        ses = 2.0
    elif (x < 15) and (x > 13):
        ses = 3.0
    elif (x < 13) and (x > 11):
        ses = 4.0
    else:
        ses = 5.0
    return ses


In [ ]:
df["SES"] = df.apply(lambda x : fill_ses(x.EDUC) if pd.isna(x.SES)   else x.SES, axis = 1)



In [ ]:
df["SES"] = df.SES.astype(int)



In [ ]:
df["MMSE"] = df.MMSE.fillna(df[df["Group"] == "Demented"]["MMSE"].median())



In [ ]:
df["Sex"] = np.where(df["Sex"] == "M", 1, 0)


In [ ]:
df.drop(["Hand", "MRI ID"], axis = 1, inplace = True)



Handling Missing Data

In [ ]:
# 11
df.loc[11,'Visit'] = 2
df.loc[12,'Visit'] = 3
df.iloc[10:13,:]


In [ ]:
# 30
df.loc[30,'Visit'] = 2
df.loc[31,'Visit'] = 3
df.loc[32,'Visit'] = 4
df.iloc[29:33,:]



In [ ]:
# 34
df.loc[34,'Visit'] = 2
df.loc[35,'Visit'] = 3
df.iloc[33:36,:]


In [ ]:
# 69
df.loc[69,'Visit'] = 2
df.loc[70,'Visit'] = 3
df.iloc[68:71]


In [ ]:
# 188
df.loc[188,'Visit'] = 2
df.iloc[187:189]


In [ ]:
# 274
df.loc[274,'Visit'] = 2
df.iloc[273:275]


In [ ]:
df_copy = df.copy()
df = df[df["Visit"] < 3]


In [ ]:
max_visit = df[df["Group"] == "Converted"].groupby(["Subject ID"]).max().reset_index()[["Subject ID","Visit"]]
for i in range(max_visit.shape[0]):
    target_index =  df[(df["Subject ID"] == max_visit.loc[i,"Subject ID"]) & (df["Visit"] != max_visit.loc[i,"Visit"]) ].index
    df.loc[target_index,"Group"] = "Nondemented"


In [ ]:
df["Group"] = np.where(df["Group"] == "Nondemented", 0, 1)



In [ ]:
split = splitter.split(df, groups=df['Subject ID'])
train_indexes, test_indexes = next(split)
train = df.iloc[train_indexes]
test = df.iloc[test_indexes]


In [ ]:
x_train = train.drop(["Subject ID", "Group"], axis = 1)
y_train = train[["Group"]]
x_test = test.drop(["Subject ID", "Group"], axis = 1)
y_test = test[["Group"]]


In [ ]:
delay_scaler = MinMaxScaler()
age_scaler = MinMaxScaler()
educ_scaler = MinMaxScaler()
mmse_scaler = MinMaxScaler()
etiv_scaler = MinMaxScaler()

delay_scaler.fit(x_train["MR Delay"].to_numpy().reshape(-1,1))
age_scaler.fit(x_train["Age"].to_numpy().reshape(-1,1))
educ_scaler.fit(x_train["EDUC"].to_numpy().reshape(-1,1))
mmse_scaler.fit(x_train["MMSE"].to_numpy().reshape(-1,1))
etiv_scaler.fit(x_train["eTIV"].to_numpy().reshape(-1,1))

x_train["MR Delay"] = delay_scaler.transform(x_train["MR Delay"].to_numpy().reshape(-1,1))
x_train["Age"] = age_scaler.transform(x_train["Age"].to_numpy().reshape(-1,1))
x_train["EDUC"] = educ_scaler.transform(x_train["EDUC"].to_numpy().reshape(-1,1))
x_train["MMSE"] = mmse_scaler.transform(x_train["MMSE"].to_numpy().reshape(-1,1))
x_train["eTIV"] = etiv_scaler.transform(x_train["eTIV"].to_numpy().reshape(-1,1))

x_test["MR Delay"] = delay_scaler.transform(x_test["MR Delay"].to_numpy().reshape(-1,1))
x_test["Age"] = age_scaler.transform(x_test["Age"].to_numpy().reshape(-1,1))
x_test["EDUC"] = educ_scaler.transform(x_test["EDUC"].to_numpy().reshape(-1,1))
x_test["MMSE"] = mmse_scaler.transform(x_test["MMSE"].to_numpy().reshape(-1,1))
x_test["eTIV"] = etiv_scaler.transform(x_test["eTIV"].to_numpy().reshape(-1,1))


In [ ]:
x_train=np.array(x_train,dtype=float)
x_test=np.array(x_test,dtype=float)
x_train=np.reshape(x_train,(int(x_train.shape[0]/2),2,11))
x_test=np.reshape(x_test,(int(x_test.shape[0]/2),2,11))


In [ ]:
y_train=np.array(y_train,dtype=float)
y_test=np.array(y_test,dtype=float)
y_train=np.reshape(y_train,(int(y_train.shape[0]/2),2,1))
y_test=np.reshape(y_test,(int(y_test.shape[0]/2),2,1))



In [ ]:
y_test.shape


Model

In [ ]:
def build_lstm_rnn():
    lstm_rnn = tf.keras.Sequential()
    lstm_rnn.add(tf.keras.layers.LSTM(100, return_sequences = True,batch_input_shape = (x_train.shape[1],2,1) ) )
    lstm_rnn.add(tf.keras.layers.LSTM(50,return_sequences = False))
    lstm_rnn.add(tf.keras.layers.Dense(1,activation = "sigmoid"))
    lstm_rnn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    return lstm_rnn


In [ ]:
lstm_rnn = build_lstm_rnn()
lstm_rnn.fit(x_train, y_train, epochs = 1)
not_final_lstm_rnn_prediction = lstm_rnn.predict(x_test)
lstm_rnn_prediction = np.where(not_final_lstm_rnn_prediction > 0.5, 1, 0)
print(classification_report(y_test,lstm_rnn_prediction, target_names= ["Non-Demanted", "Demanted"]))
lstm_rnn_conf = confusion_matrix(y_test,lstm_rnn_prediction)
lstm_rnn_plot_conf = ConfusionMatrixDisplay(lstm_rnn_conf)
lstm_rnn_plot_conf.plot()
